In [2]:
import json
from pathlib import Path
import re
import json
import random

In [3]:
path = Path("../../data/gqa/data/val_all_questions.json")

In [4]:
with open(path) as f:
    question = json.load(f)
random.seed(42)
keys = list(question.keys())
subset_size = int(0.1 * len(keys))
subset_keys = random.sample(keys, subset_size)
subset = {key: question[key] for key in subset_keys}

In [5]:
print(len(question))

2011853


In [6]:
# print(subset_keys[:10])
# for item in subset_keys[:10]:
#     print(subset[item])
#     print("\n")

In [7]:
for key, val in subset.items():
    print(val)
    print(val['types']['semantic'])
    break

{'semantic': [{'operation': 'select', 'dependencies': [], 'argument': 'table (1276780)'}, {'operation': 'exist', 'dependencies': [0], 'argument': '?'}, {'operation': 'select', 'dependencies': [], 'argument': 'plate (-) '}, {'operation': 'exist', 'dependencies': [2], 'argument': '?'}, {'operation': 'or', 'dependencies': [1, 3], 'argument': ''}], 'entailed': [], 'equivalent': ['03716212'], 'question': 'Are there any plates or tables in the scene?', 'imageId': '2387346', 'isBalanced': False, 'groups': {'global': None, 'local': '09existOr-plate_table'}, 'answer': 'yes', 'semanticStr': 'select: table (1276780)->exist: ? [0]->select: plate (-) ->exist: ? [2]->or:  [1, 3]', 'annotations': {'answer': {}, 'question': {'5': '1276780'}, 'fullAnswer': {'4': '1276780'}}, 'types': {'detailed': 'existOr', 'semantic': 'obj', 'structural': 'logical'}, 'fullAnswer': 'Yes, there is a table.'}
obj


In [8]:
print(len(question))

2011853


# first round of filtering

In [10]:
# first filter
filtered_single_noun_dic = {}
filtered_more_nouns_dic = {}
multiple_mention_dic = {}
half_or_no_mention_dic = {}
global_dic = {}
noun_not_in_q = {}
max_noun_num = 0

pattern = r"(\w+)\s*\((\d+(?:\s*,\s*\d+)*)\)" # should be a correct one
for key, val in subset.items():
    question = val['question']
    question_type = val['types']['semantic']
    nouns = []
    if question_type != 'global':
        overall_flag = True
        multiple_mention_flag = False
        half_or_no_mention_flag = False
        noun_not_in_q_flag = False
        semantics = val.get('semantic', [])
        for operation in semantics:
            if operation['operation'] == 'select':
                text = operation['argument']
                match = re.search(pattern, text)
                if match: 
                    noun = match.group(1)
                    if noun not in question: 
                        overall_flag = False
                        noun_not_in_q_flag = True
                    else:
                        numbers = match.group(2)
                        number_list = [n.strip() for n in numbers.split(',')]
                        if len(number_list) > 1:
                            # as long as there is a multiple mention, add this case
                            multiple_mention_flag = True
                            overall_flag = False
                        nouns.append(noun)
                else: 
                    overall_flag = False
                    half_or_no_mention_flag = True
        if len(nouns) > max_noun_num:
            max_noun_num = len(nouns)
        if overall_flag:
            if len(nouns) > 1:
                filtered_more_nouns_dic[key] = val
            else:
                filtered_single_noun_dic[key] = val
        elif multiple_mention_flag:
            multiple_mention_dic[key] = val
        elif half_or_no_mention_flag:
            half_or_no_mention_dic[key] = val  
        elif noun_not_in_q_flag:
            noun_not_in_q[key] = val
    else:
        global_dic[key] = val        

In [11]:
print(len(subset))
print(len(filtered_single_noun_dic))
print(len(filtered_more_nouns_dic))
print(len(multiple_mention_dic))
print(len(half_or_no_mention_dic))
print(len(global_dic))
print(len(noun_not_in_q))

201185
133986
7377
207
51617
3128
4870


In [15]:
141363 - 133986

7377

In [16]:
final = len(subset) - (len(filtered_single_noun_dic) + len(multiple_mention_dic) + len(half_or_no_mention_dic) + len(global_dic) + len(noun_not_in_q))
final

7377

In [18]:
print(max_noun_num)

2


In [19]:
# store as json file
with open('../data/filtered_single_nouns.json', 'w') as f1:  # 'wb' means write in binary mode
    json.dump(filtered_single_noun_dic, f1, indent=4)
with open('../data/filtered_more_nouns.json', 'w') as f2:  # 'wb' means write in binary mode
    json.dump(filtered_more_nouns_dic, f2, indent=4)
with open('../data/multiple_references_dic.json', 'w') as f3:  # 'wb' means write in binary mode
    json.dump(multiple_mention_dic, f3, indent=4)
with open('../data/half_mention.json', 'w') as f4:  # 'wb' means write in binary mode
    json.dump(half_or_no_mention_dic, f4, indent=4)
with open('../data/global_questions.json', 'w') as f5:  # 'wb' means write in binary mode
    json.dump(global_dic, f5, indent=4)
with open('../data/noun_not_in_q.json', 'w') as f6:  # 'wb' means write in binary mode
    json.dump(noun_not_in_q, f6, indent=4)  

# Hypernym replacement

In [3]:
# load the filtered_dic
with open("../data/filtered_single_nouns.json", 'r') as f:
    filtered_single_noun_dic = json.load(f)

In [4]:
# read the hypernym file first
with open('../data/gqa_entities/noun-hypernyms.json') as f:
    hyper = json.load(f)
print(len(hyper))

933


In [12]:
len(filtered_single_noun_dic)

133986

In [27]:
pattern = r"(\w+)\s*\((\d+(?:\s*,\s*\d+)*)\)" # should be a correct one
replaced_dic = {}
no_hypernym = {}
for key, val in filtered_single_noun_dic.items():
    question_str = val['question']
    semantics = val.get('semantic', [])
    question_dic = {}
    for operation in semantics:
        if operation['operation'] == 'select':
            text = operation['argument']
            match = re.search(pattern, text)
            noun = match.group(1)
    hypers = hyper.get(noun, None) #["adult", "person"]
    if hypers is not None: 
        new_qs = []
        for hyp in hypers:
            new_q = question_str.replace(noun, hyp)
            new_qs.append(new_q)
        question_dic['original'] = question_str
        question_dic['new'] = new_qs
        question_dic['arg'] = noun
        question_dic['hypernym'] = hypers
        question_dic['answer'] = val['answer']
        question_dic['fullAnswer'] = val['fullAnswer']
        question_dic['imageId'] = val['imageId']
    if len(question_dic) > 0:
        replaced_dic[key] = question_dic
    else: 
        no_hypernym[key] = val

In [13]:
hyper.get('man') # just save the information in the json. 

['adult', 'person']

In [40]:
keys = list(no_hypernym.keys())[:5]

In [46]:
# for key, val in no_hypernym.items():
#     print(val)
#     break
for key in keys:
    print(no_hypernym[key])
    print("\n")
res = hyper.get('palm trees')
print(res)

{'semantic': [{'operation': 'select', 'dependencies': [], 'argument': 'palm trees (1144319)'}, {'operation': 'relate', 'dependencies': [0], 'argument': 'traffic light,to the left of,s (1144320)'}, {'operation': 'choose color', 'dependencies': [1], 'argument': 'black|orange'}], 'entailed': [], 'equivalent': ['03959891'], 'question': 'Which color is the signal light to the left of the palm trees, black or orange?', 'imageId': '2401770', 'isBalanced': False, 'groups': {'global': 'color', 'local': '10c-traffic light_color'}, 'answer': 'black', 'semanticStr': 'select: palm trees (1144319)->relate: traffic light,to the left of,s (1144320) [0]->choose color: black|orange [1]', 'annotations': {'answer': {}, 'question': {'11:13': '1144319', '4:6': '1144320'}, 'fullAnswer': {'1:3': '1144320'}}, 'types': {'detailed': 'chooseAttr', 'semantic': 'attr', 'structural': 'choose'}, 'fullAnswer': 'The traffic signal is black.'}


{'semantic': [{'operation': 'select', 'dependencies': [], 'argument': 'shel

In [43]:
for key, val in replaced_dic.items():
    print(val)
    break

{'original': 'Is there a plate to the left of the man that is on the right side?', 'new': ['Is there a plate to the left of the adult that is on the right side?', 'Is there a plate to the left of the person that is on the right side?'], 'arg': 'man', 'hypernym': ['adult', 'person'], 'answer': 'no', 'fullAnswer': 'No, there is a bottle to the left of the man.', 'imageId': '2393739'}


In [15]:
print(len(replaced_dic))

86390


In [16]:
with open('../data/replacement.json', 'w') as f7:  
     json.dump(replaced_dic, f7, indent=4)

In [18]:
# load the scenegraph
def load_scenegraphs(split):
    path = f"../data/sceneGraphs/{split}_sceneGraphs.json"
    with open(path) as f:
        d = json.load(f)
    return d

In [19]:
val_scenegraphs = load_scenegraphs("val")

In [21]:
final_dic = {}
num_not_match_dic = {}
dup_obj_dic = {}
none_num = 0
for key, val in replaced_dic.items():
    image_id = val.get('imageId')
    item = val_scenegraphs.get(image_id, None)
    if item is not None:
        # if len(item['objects']) >=2 and len(item['objects']) <= 20:
        if 2 <= len(item['objects']) <=20:
            all_names = [obj['name'] for obj in item['objects'].values()]
            duplicated_flag = len(set(all_names))!=len(all_names)
            # name_set = set()
            # duplicated_flag = False
            # for key_1, objs in item['objects'].items():
            #     if objs['name'] not in name_set:
            #         name_set.add(objs['name'])
            #     else:
            #         duplicated_flag = True
            if not duplicated_flag:
                final_dic[key] = val
            else:
                dup_obj_dic[key] = val
            # final_dic[key] = val
        else:
            num_not_match_dic[key] = val  
    else:
        none_num += 1

In [22]:
print(len(final_dic))

15762


In [23]:
print(len(num_not_match_dic))

33439


In [24]:
print(len(dup_obj_dic))
print(len(dup_obj_dic)+len(final_dic))

37189
52951


In [25]:
print(none_num)

0


In [26]:
print(len(replaced_dic))

86390


In [89]:
len(final_dic) / (len(final_dic) + len(else_dic) + none_num)

0.39519800576179603

In [73]:
len(else_dic) / (len(final_dic) + len(else_dic) + none_num)

0.24949621602256952

In [66]:
none_num / (len(final_dic) + len(else_dic) + none_num)

0.35523114355231145

In [113]:
# for key, val in val_scenegraphs.items():
#     duplicated_flag = False
#     name_set = ()
#     for item in val['objects']:
#         if item['name'] not in name_set:
#             name_set.add(item['name'])
#         else
#             duplicated_flag = True
#             break
#     if not duplicated_flag:
for key, val in final_dic.items():
    print(val)
    break
    

{'original': 'Is there any cup to the left of the napkin?', 'new': ['Is there any cup to the left of the table linen?', 'Is there any cup to the left of the linen?', 'Is there any cup to the left of the clothing?'], 'arg': 'napkin', 'hypernym': ['table linen', 'linen', 'clothing'], 'answer': 'yes', 'fullAnswer': 'Yes, there is a cup to the left of the napkin.', 'imageId': '2347577'}


In [145]:
with open("../data/val_final_filtered_questions.json", 'w') as f8:
    json.dump(final_dic, f8, indent=4)